In [1]:
import sys
if "../src/" not in sys.path:
    sys.path.insert(0,"../src/")

In [11]:
import numpy as np
from pbcpy.formats.qepp import PP
from pbcpy.optimization import Optimization
from pbcpy.functionals import FunctionalClass, TotalEnergyAndPotential

### load guess density

In [3]:
mol = PP(filepp='Al_fde_rho.pp').read()

PP_file not set in input. Can do so manually invoking Atom.local_PP
PP_file not set in input. Can do so manually invoking Atom.local_PP


### Load KEDF

In [4]:
optional_kwargs = {}
optional_kwargs["Sigma"] = 0.0
optional_kwargs["x"] = 1.0
optional_kwargs["y"] = 1.0
KE = FunctionalClass(type='KEDF',name='x_TF_y_vW',is_nonlocal=False,optional_kwargs=optional_kwargs)

### load XC

In [5]:
XC = FunctionalClass(type='XC',name='LDA',is_nonlocal=False)

### load IONS and HARTREE

In [6]:
optional_kwargs = {}
optional_kwargs["PP_list"] = ["./Al_lda.oe01.recpot","./Al_lda.oe01.recpot"]
optional_kwargs["ions"]    = mol.ions 

IONS = FunctionalClass(type='IONS', optional_kwargs=optional_kwargs)
HARTREE = FunctionalClass(type='HARTREE')

### load energy evaluator

In [7]:
E_v_Evaluator = TotalEnergyAndPotential(rho=mol.field,
                                        KineticEnergyFunctional=KE,
                                        XCFunctional=XC,
                                        HARTREE=HARTREE,
                                        IONS=IONS)

### instance optimizer

In [8]:
opt = Optimization(EnergyEvaluator=E_v_Evaluator)

### optimize!

In [9]:
new_rho = opt.get_optimal_rho(guess_rho=mol.field)

Recpot pseudopotential ./Al_lda.oe01.recpot loaded
Recpot pseudopotential ./Al_lda.oe01.recpot loaded
b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'


In [12]:
delta_rho = np.abs(new_rho - mol.field).integral()/2

In [13]:
print(delta_rho)

0.261357987978
